In [1]:
%reset -f

In [2]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
dfAll =[]

pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    ddf = pd.read_pickle(os.path.join('resources','results', str(f) ))
print ddf.shape
dfAll.append(ddf)

(5, 26)


In [13]:
dfs = pd.DataFrame(np.vstack(dfAll), columns=ddf.columns)

dfs.cv_f1_mean = pd.to_numeric(dfs.cv_f1_mean)
dfs.cv_rec_mean = pd.to_numeric(dfs.cv_rec_mean)
dfs.cv_prec_mean = pd.to_numeric(dfs.cv_prec_mean)

dfs.cv_f1_std = pd.to_numeric(dfs.cv_f1_std)
dfs.cv_prec_std = pd.to_numeric(dfs.cv_prec_std)
dfs.cv_rec_std = pd.to_numeric(dfs.cv_rec_std)
dfs.exp = pd.to_numeric(dfs.exp)

dfs["cv_f1"] = dfs["cv_f1_mean"].round(2).astype(str).str.cat(dfs["cv_f1_std"].round(2).astype(str), sep="+/-")
dfs["cv_prec"] = dfs["cv_prec_mean"].round(2).astype(str).str.cat(dfs["cv_prec_std"].round(2).astype(str), sep="+/-")
dfs["cv_rec"] = dfs["cv_rec_mean"].round(2).astype(str).str.cat(dfs["cv_rec_std"].round(2).astype(str), sep="+/-")
dfs["num_nones"] = np.sum(np.hstack(((dfs.sm == "none").reshape(-1,1),(dfs.fs == "none").reshape(-1,1))), axis=1)

print dfs.columns.tolist()
print dfs.shape

['exp', 'typeDisease', 'typeEncounter', 'typeHypothesis', 'typeDataFeatures', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_f1', 'tr_prec', 'tr_rec', 'cv_f1_mean', 'cv_f1_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'test_f1', 'test_prec', 'test_rec', 'test_auc', 'time', 'pipeline', 'cv_f1', 'cv_prec', 'cv_rec', 'num_nones']
(5, 30)


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app


In [14]:
dfs.sort_values(["cv_f1_mean","exp"], ascending=False,inplace=True)
dfs[["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
3,0.10,none,recall,rf,"[entropy, 4, balanced, 150]",0.65,0.65,0.64,0.63,0.63,0.63,0.60,0.60,0.61,0.59
4,0.10,none,roc_auc,svmRBF,"[0.001, balanced, 50]",0.65,0.65,0.65,0.63,0.63,0.63,0.61,0.61,0.61,0.59
0,0.10,none,roc_auc,logReg,"[balanced, 0.1, l2]",0.62,0.63,0.62,0.62,0.63,0.62,0.60,0.60,0.61,0.59
1,0.10,none,recall,logReg,"[balanced, 1e-05, l2]",0.60,0.62,0.60,0.61,0.62,0.60,0.58,0.58,0.59,0.58
2,0.10,none,roc_auc,rf,"[gini, 6, None, 250]",0.63,0.72,0.68,0.58,0.64,0.64,0.56,0.63,0.62,0.55


In [15]:
np.sum(dfs["time"])

693.3270304203033